In [1]:
import ultralytics
from ultralytics import YOLO
import os
import pytesseract
import cv2
import numpy

In [2]:
HOME = os.getcwd()
HOME

'/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/receipt extraction model.v2i.yolov8'

In [19]:
model = YOLO('trained_model/best.pt')
print(type(model))

<class 'ultralytics.models.yolo.model.YOLO'>


In [4]:
def extract_text(image : numpy.ndarray):
    """
    Extract text from an image using Tesseract OCR.
    """
    # Read the image
    # image = cv2.imread(image_path)

    # Convert the image to gray scale
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Use Tesseract to do OCR on the image
    text = pytesseract.image_to_string(gray_image)

    return text


In [6]:
import cv2

# Load the image
file_path = "/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/receipt extraction model.v2i.yolov8/train/images/0676aa86-IMG_20231221_163400_jpg.rf.5ce8e6bcbbfebbb12a6ecffb602abe3f.jpg"

# Run your model
results = model(file_path)

# Dictionary to store highest confidence detection for each class
highest_conf_detections = {}
for result in results:
    for box in result.boxes:
        class_label = result.names[int(box.cls)]
        conf = box.conf

        # Check if this class_label already exists and if this detection has higher confidence
        if class_label not in highest_conf_detections or highest_conf_detections[class_label].conf < conf:
            highest_conf_detections[class_label] = box

# Now process only the detections with highest confidence for each class
for class_label, box in highest_conf_detections.items():
    image = cv2.imread(file_path)

    # Get box coordinates
    x1 = int(box.xyxy[0][0])
    y1 = int(box.xyxy[0][1])
    x2 = int(box.xyxy[0][2])
    y2 = int(box.xyxy[0][3])

    print(x1, y1, x2, y2)

    # Draw rectangle (bounding box)
    cv2.rectangle(image, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2)

    # Put class label with background rectangle
    label_size, base_line = cv2.getTextSize(class_label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
    y1_label = max(y1, label_size[1] + 10)
    cv2.rectangle(image, (x1, y1 - label_size[1] - 10), (x1 + label_size[0], y1), (0, 255, 0), cv2.FILLED)
    cv2.putText(image, class_label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    # Save or show the result
    # cv2.imwrite(f"/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/receipt extractor/experimentation/receipt extraction model.v2i.yolov8/annotion_extraction_test/annotated_image_{class_label}.jpg", image)
    # # cropped the picture
    crop_img = image[y1:y2, x1:x2]
    # cv2.imwrite(f"/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/receipt extractor/experimentation/receipt extraction model.v2i.yolov8/annotion_extraction_test/annotated_image_{class_label}_cropped.jpg", crop_img)

    print(f"\n\n {class_label} \n\n")
    print(extract_text(crop_img))



image 1/1 /Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/receipt extraction model.v2i.yolov8/train/images/0676aa86-IMG_20231221_163400_jpg.rf.5ce8e6bcbbfebbb12a6ecffb602abe3f.jpg: 640x640 1 item_purshase, 510.4ms
Speed: 2.0ms preprocess, 510.4ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)
150 101 490 409


 item_purshase 


eR MART ie

recension
INTERMART Grand Baie Le Crotset le
Paltoni Retall Ld
T/A: VA12724306
SRN: 613419389
VAT INVOICE

22/11/2028 17:96:50 1aZ6BL¢
INT ONIONS RED KG R880 Tx
ANT CAROTTE LOZAL KG Rs4Z.24 Tx:
LABEL 60 COEUR POULET SURG 2'0

2 @ Rs24.85ea Rs49,70 [x2
S1 ELOZ PUREE FOMATE BRK SOOI L

2 @ Rs59.95ea, Rs119.90 Txt
CUISTO CHK MINS 400ch

8 8 Rs74.95ea, Rs599.60 Tx2
PRO CHICKEN MEYCED 4006

2H Rst.95ea. Rs169.90 Tx2

susp TOTAL Rs390_.14




In [24]:
def extract_receipt(model:YOLO, file_path):
    # Run your model
    results = model(file_path)

    receipt_data = {}

    # Dictionary to store highest confidence detection for each class
    highest_conf_detections = {}
    for result in results:
        for box in result.boxes:
            class_label = result.names[int(box.cls)]
            conf = box.conf

            # Check if this class_label already exists and if this detection has higher confidence
            if class_label not in highest_conf_detections or highest_conf_detections[class_label].conf < conf:
                highest_conf_detections[class_label] = box

    # Now process only the detections with highest confidence for each class
    for class_label, box in highest_conf_detections.items():
        image = cv2.imread(file_path)

        # Get box coordinates
        x1 = int(box.xyxy[0][0])
        y1 = int(box.xyxy[0][1])
        x2 = int(box.xyxy[0][2])
        y2 = int(box.xyxy[0][3])

        crop_img = image[y1:y2, x1:x2]

        receipt_data[class_label] = extract_text(crop_img)
    return receipt_data


In [25]:

# Load the image
# file_path = "/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/receipt extractor/experimentation/receipt extraction model.v2i.yolov8/train/images/0676aa86-IMG_20231221_163400_jpg.rf.5ce8e6bcbbfebbb12a6ecffb602abe3f.jpg"
file_path = "/Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/Server/uploads/CAP2767703974190339483.jpg"
print(extract_receipt(model, file_path))


image 1/1 /Users/damienmaujean/Library/CloudStorage/OneDrive-Personal/university/MiddleSex University/Year 3/CST3990 Individual Project/src/Server/uploads/CAP2767703974190339483.jpg: 640x384 (no detections), 274.9ms
Speed: 1.5ms preprocess, 274.9ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)
{}
